In [1]:
#Dependencies
import pandas as pd
import requests
import json
from config import api_key


In [2]:
states_abbrev_file_path = "Resources/states_abbrev.csv"

states_abbrev_df = pd.read_csv(states_abbrev_file_path)
states_abbrev_df

,State,Abbrev
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [3]:
# API Call for covid data 
covid_base_url = "https://api.covidtracking.com/v1/states/daily.json"
response = requests.get(covid_base_url).json()

#print(json.dumps(response, indent=4))

# pull the data from a source df to a target df by creating a new df

covid_df = pd.DataFrame(response, columns=['date', 'state', 'positive', 'death', 'hospitalized', 'totalTestResults'])
covid_df.head()

,date,state,positive,death,hospitalized,totalTestResults
0,20201018,AK,12016.0,67.0,NaN,532711.0
1,20201018,AL,172626.0,2788.0,18855.0,1253896.0
2,20201018,AR,99066.0,1704.0,6315.0,1215944.0
3,20201018,AS,0.0,0.0,NaN,1616.0
4,20201018,AZ,231149.0,5827.0,20640.0,1631023.0


In [4]:
for x, y in covid_df.iterrows():
    org_date = str(y.get(key='date'))
   
    year_s = org_date[0:4]
    month_s = org_date[4:6]
    day_s = org_date[6:8]
    formatted_date = f"{year_s}-{month_s}-{day_s}"
    
        
covid_df['Formatted Date'] = formatted_date
covid_df.head()

        

,date,state,positive,death,hospitalized,totalTestResults,Formatted Date
0,20201018,AK,12016.0,67.0,NaN,532711.0,2020-01-22
1,20201018,AL,172626.0,2788.0,18855.0,1253896.0,2020-01-22
2,20201018,AR,99066.0,1704.0,6315.0,1215944.0,2020-01-22
3,20201018,AS,0.0,0.0,NaN,1616.0,2020-01-22
4,20201018,AZ,231149.0,5827.0,20640.0,1631023.0,2020-01-22


In [5]:
# API Call for flights data 

url = f"http://api.aviationstack.com/v1/flights?access_key={api_key}&"
query_date = ['2020-10-01','2020-10-02','2020-10-03' ]
query_iata = ['JFK', 'SAN']
value_list = []

for q_date in query_date:
    for q_iata in query_iata:
        flights_base_url = f"{url}flight_date={q_date}&arr_iata={q_iata}&flight_status=landed&limit=1"
        flights_response = requests.get(flights_base_url).json()
        landed_flights = flights_response['pagination']['total']
        state = flights_response['data'][0]['arrival']['timezone']
        state = state.split('/')[1]
        state = state.replace('_', ' ')
       
        value_list.append({'Flight Date': q_date, 'State': state, 'IATA':q_iata, 'Total Landed Flights': landed_flights})
        
flights_df = pd.DataFrame(value_list)   
flights_df


,Flight Date,State,IATA,Total Landed Flights
0,2020-10-01,New York,JFK,840
1,2020-10-01,Los Angeles,SAN,210
2,2020-10-02,New York,JFK,821
3,2020-10-02,Los Angeles,SAN,206
4,2020-10-03,New York,JFK,744
5,2020-10-03,Los Angeles,SAN,192


In [7]:
#df1.merge(df2,on='name').merge(df3,on='name')
merged_df = states_abbrev_df.merge(flights_df, on='State')
merged_df

,State,Abbrev,Flight Date,IATA,Total Landed Flights
0,New York,NY,2020-10-01,JFK,840
1,New York,NY,2020-10-02,JFK,821
2,New York,NY,2020-10-03,JFK,744
